In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
class_name = 'chair'

do_jsd = True
do_mmd = False
do_cov = False

mmd_loss = 'chamfer'

skip = ['train', 'test']

boost_samples = 3
n_reps = 3
subsample = True # new - for rebuttal to get entire dataset

In [20]:
if do_jsd:
    boost_samples = 1
else:
    boost_samples = boost_samples
assert(np.sum([do_cov, do_jsd, do_mmd]) == 1)

In [21]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [23]:
if do_mmd:
    func_ = evaluator.compute_mmd    
elif do_cov:
    func_ = evaluator.compute_coverage    

In [24]:
train_data = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/chair_train.npz'
sample_files = [train_data]

In [25]:
trials_res = []
for _ in range(n_reps):
    for sample_file in sample_files:
        evaluator.prepare_sample_data(sample_file, boost_samples)
        
        if not subsample: # new for rebuttal
            evaluator.sample_data['test'] = evaluator.sample_data['train']
            evaluator.sample_data['val'] = evaluator.sample_data['train']
        
        sample_name = osp.basename(sample_file)[:-len('.npz')]
        
        print sample_name

        if do_jsd:
            res = evaluator.compute_jsd()
        else:            
            res = func_(loss=mmd_loss, sample_estimator=False, skip=skip)
            
        trials_res.append(res)    

chair_train
train 0.0
test 0.010702205945
val 0.0156324837713
chair_train
train 0.0
test 0.0100690436088
val 0.0167675473155
chair_train
train 0.0
test 0.0102530717837
val 0.0186365192882


In [26]:
split = 'val'
avgs = []
for i in range(n_reps):
    avgs.append(np.mean(trials_res[i][split]))
print avgs
print np.mean(avgs)

[0.015632483771256034, 0.016767547315504672, 0.018636519288238063]
0.0170121834583
